# Hierarchical Agent System with LangGraph

This notebook demonstrates a hierarchical agent system where a **Router Agent** analyzes user input (containing job requirements and a task) and routes it to either a **Resume Maker** or a **Cover Letter Writer**.

In [1]:
%pip install -U langgraph langchain langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.80
    Uninstalling langchain-core-0.3.80:
      Successfully uninstalled langchain-core-0.3.80
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.27
    Uninstalling langchain-0.3.27:
      Successfully uninstalled langchain-0.3.27


In [2]:
import os
import getpass

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

KeyboardInterrupt: Interrupted by user

In [ ]:
from typing import TypedDict, Literal
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END

# 1. Define State
class AgentState(TypedDict):
    messages: list
    next_step: Literal["resume_check", "resume_maker", "cover_letter_writer", "FINISH"]
    user_input: str
    final_output: str

In [ ]:
# Setup LLM
llm = ChatGroq(model="openai/gpt-oss-120b", temperature=0.7)

In [ ]:
# 2. Define Router Agent
def router_node(state: AgentState):
    print("--- ROUTER AGENT ---")
    user_input = state["user_input"]

    system_prompt = (
        "You are a helpful router agent. Your job is to analyze the user's input, "
        "which contains job requirements and a specific task request. "
        "Determine if the user wants a 'Resume' or a 'Cover Letter'. "
        "Return ONLY the word 'resume' or 'cover_letter'."
    )

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_input)
    ]

    response = llm.invoke(messages)
    decision = response.content.strip().lower()

    if "resume" in decision:
        return {"next_step": "resume_check"} # Route to check first
    elif "cover" in decision:
        return {"next_step": "cover_letter_writer"}
    else:
        return {"next_step": "FINISH", "final_output": "Could not determine intent."}

In [ ]:
# 3. Define Specialized Agents

def resume_check_node(state: AgentState):
    print("--- RESUME CHECK AGENT ---")
    user_input = state["user_input"]

    prompt = (
        "You are a Resume Quality Checker. Analyze the following user input. "
        "Check if it contains sufficient information for: 1. Contact Info, 2. Skills, 3. Experience, 4. Education. "
        "If ANY of these are missing, list what is missing and ask the user to provide it. "
        "If ALL are present, return ONLY the word 'PROCEED'.\n\n"
        f"Input: {user_input}"
    )

    response = llm.invoke([HumanMessage(content=prompt)])
    content = response.content.strip()

    if "PROCEED" in content.upper():
        return {"next_step": "resume_maker"}
    else:
        return {"final_output": content, "next_step": "FINISH"}

def resume_maker_node(state: AgentState):
    print("--- RESUME MAKER AGENT ---")
    user_input = state["user_input"]

    prompt = (
        "You are an expert Resume Writer. "
        "Based on the following job requirements and user details, create a professional resume. "
        "Do not add anything if user has not given, if any section is missing ask from user and then only create the resume.\n\n"
        f"Input: {user_input}"
    )

    response = llm.invoke([HumanMessage(content=prompt)])
    return {"final_output": response.content, "next_step": "FINISH"}

def cover_letter_writer_node(state: AgentState):
    print("--- COVER LETTER WRITER AGENT ---")
    user_input = state["user_input"]

    prompt = (
        "You are an expert Cover Letter Writer. "
        "Based on the following job requirements and user details, write a compelling cover letter. "
        "Do not make things on your own, cover letter should be as per the user details.\n\n"
        f"Input: {user_input}"
    )

    response = llm.invoke([HumanMessage(content=prompt)])
    return {"final_output": response.content, "next_step": "FINISH"}

In [ ]:
# 4. Build Graph
workflow = StateGraph(AgentState)

workflow.add_node("router", router_node)
workflow.add_node("resume_check", resume_check_node)
workflow.add_node("resume_maker", resume_maker_node)
workflow.add_node("cover_letter_writer", cover_letter_writer_node)

workflow.set_entry_point("router")

def route_decision(state: AgentState):
    return state["next_step"]

workflow.add_conditional_edges(
    "router",
    route_decision,
    {
        "resume_check": "resume_check",
        "cover_letter_writer": "cover_letter_writer",
        "FINISH": END
    }
)

workflow.add_conditional_edges(
    "resume_check",
    route_decision,
    {
        "resume_maker": "resume_maker",
        "FINISH": END
    }
)

workflow.add_edge("resume_maker", END)
workflow.add_edge("cover_letter_writer", END)

app = workflow.compile()

In [ ]:
# 5. Execute Graph

def run_agent(user_input):
    initial_state = {"user_input": user_input, "messages": []}
    result = app.invoke(initial_state)
    return result.get("final_output")

In [ ]:
# Example 1: Incomplete Resume Request
input_1 = """
I need a resume for a Senior Python Developer role.
Requirements: 5+ years of Python, Django, FastAPI, AWS experience.
My details: Ajeet, 6 years exp in Python.
"""
print("--- Test 1: Incomplete Input ---")
print(run_agent(input_1))

In [ ]:
# Example 2: Complete Resume Request
input_2 = """
I need a resume for a Senior Python Developer role.
Requirements: 5+ years of Python, Django, FastAPI, AWS experience.
My details:
Name: Ajeet Kumar
Contact: ajeet@example.com, 123-456-7890
Skills: Python, Django, FastAPI, AWS, Docker
Experience: 6 years at Tech Corp as Senior Dev. Built scalable APIs.
Education: B.Tech in CS from XYZ University.
"""
print("\n--- Test 2: Complete Input ---")
print(run_agent(input_2))

In [ ]:
# Example 3: Cover Letter Request
input_3 = """
Write a cover letter for a Frontend Engineer position at Google.
Requirements: React, TypeScript, 3 years exp.
My details: Ajeet, passionate about UI/UX, expert in React.
"""
print("\n--- Test 3: Cover Letter ---")
print(run_agent(input_3))